<a href="https://colab.research.google.com/github/Bateyjosue/Hackerthon_NLP_Fellowship/blob/main/Copy_of_Josue_Batey_Hackathon_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Write a program which takes array of urls for job-posting websites in Rwanda
   * https://www.jobinrwanda.com/  (Job Adverts)
   * https://www.umucyo.gov.rw  ( Tenders)
2. Web-scrap the content and put them in a pandas data-frame(i.e: the best structured dataframe wins ofcourse!)
3. Use regular expression and key-words to retrieve IT/Software related job/consultancy opportunities(Tenders).
4. Use EasyNMT to display the results in 50+ languages using wrapped state of art models
   * Practical Example for EasyNMT use:
     https://colab.research.google.com/drive/1X47vgSiOphpxS5w_LPtjQgJmiSTNfRNC?usp=sharing
5. Hosting of our work example in colab
   * https://colab.research.google.com/drive/1kAh_Vt1ipA5-BuoaPX39rCIHFrhpcRpW?usp=sharing

## Import Packages

In [ ]:
import requests
from bs4 import BeautifulSoup

## Class For Scrapping

In [ ]:
class Web_Scrap:
  def __init__(self, link):
    self.url = link
  
  def get_content(self):
    content = requests.get(url = self.url).content
    content = BeautifulSoup(content, 'lxml')
    return content
  
  def get_content_class(self, class_, tag = None):
    content = self.get_content()
    content = content.find_all(class_= class_) if tag == None else content.find_all(tag, class_= class_)
    return content

# Scrap `job in rwanda` and `umucyo`

# Challenges #1



1.   Get `Jobs and Consultacies links` for `job in rwanda`




##Links of Jobs

In [ ]:
scrap_jobs =  Web_Scrap('https://www.jobinrwanda.com/')

In [ ]:
jobs_content = scrap_jobs.get_content()
jobs_links = []
links = scrap_jobs.get_content_class('card-title', 'h5')
for li in links:
  jobs_links.append('https://www.jobinrwanda.com' + li.parent['href'])


In [ ]:
len(jobs_links)

168

ℹ `jobs_links` Holds the list of all jobs links. you can check the lenght by typing `len(jobs_links)`


## 2.   Get `tenders` links for `umucyo`



In [ ]:
# scrap_tender = Web_Scrap('https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do')
# tender_table = scrap_tender.get_content_class('article_table mb10')

# tr = tender_table.find_all('tr')
# len(tr)

In [ ]:
tenders_links = []

3. combined links for both `job in rwand` and `umucyo`


In [ ]:
combined_links = []

#Challenge #2

## 1. Get Content per Link by Web scrapping

In [ ]:
# Help Funtions
def remove_space_list(arr):
  joined = []
  for el in arr:
    if el.strip:
      joined.append(el.strip())
  return joined

def clean_data(strs):
  import re
  text = re.sub('\W', ' ', strs)

  return text.strip()

## 2. Save Content in a DataFrame

In [ ]:
import random
import pandas as pd
frames = pd.DataFrame()
def data(link):
  dict_summary = {}
  scrap = Web_Scrap(link)
  main = scrap.get_content().body.main
  sent  = [sent for sent in main.get_text().split('\n') if sent.strip()]
  aside = scrap.get_content().body
  aside = aside.select_one('div#block-jobsummaryblock')
  for a in aside.select('ul.list-group.list-group-flush > li > b'):
    val = a.next_sibling.strip().split(',')
    dict_summary[clean_data(a.get_text().strip())] = ' '.join(remove_space_list(val)) if len(val) <= 2 else ', '.join(remove_space_list(val))
  return dict_summary

dicts = []
for job in jobs_links:
  dicts.append(data(job))

In [ ]:
col = dicts[0].keys()
values = []
for d in dicts:
  values.append(d.values())
index = [i for i, v in enumerate(values)]

f = pd.DataFrame([value if value else 'Not Specified' for value in values ], columns=col)
f

,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions
0,RW,"Accounting, Finance and investment, Other",Bachelor,Mid career (3 to 5 years of experience),Full-time,Sunday 20/11/2022 23:59,1
1,RW,Other,Bachelor,Senior (5+ years of experience),Contract,Monday 21/11/2022 09:00,1
2,RW,Other,Not specified,Not specified,Contract,Friday 25/11/2022 10:00,1
3,RW,Other,Not specified,Not specified,Tender,Monday 21/11/2022 23:59,1
4,RW,Other,Not specified,Contract,Thursday 08/12/2022 12:00,1,None
...,...,...,...,...,...,...,...
163,Kigali RW,"Agriculture, Agronomy, Business, Management",,Full-time,Sunday 20/11/2022 23:59,1,None
164,Kigali RW,"Agriculture, Agronomy, Business, Environmental...",Master,Senior (5+ years of experience),Full-time,Monday 28/11/2022 23:59,1
165,Kigali RW,"Agriculture, Business, Computer and IT, Manage...",Not specified,Not specified,Full-time,Tuesday 06/12/2022 23:59,1
166,Kigali RW,"Agriculture, Agronomy, Business, Environmental...",,Full-time,Tuesday 29/11/2022 23:59,1,None


## Remove Unclear Rows to make the df readable ✅

In [ ]:
def get_job_info(link):
  scrap = Web_Scrap(link)
  cont= scrap.get_content().select("div.card-body.p-2")
  return [[clean_data(i) for i in con.get_text().strip().split('\n') if i.strip()] for con in cont]

data = get_job_info('https://www.jobinrwanda.com/')

new_col = ['Job Title', 'Company','Published Date','Deadline','Desired experience','Job Category']
index = [i for i, v in enumerate(data)]
df = pd.DataFrame(data, index=index, columns=new_col)
df.pop('Deadline')
df.pop('Desired experience')
f.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

## Concat the dataframe to get the one with all needed data

In [ ]:
frame = pd.concat([df, f], axis='columns')
frame.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
frame = frame.reset_index(drop=True)

frame

,Job Title,Company,Published Date,Job Category,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions
0,Internal Audit Manager,Job in Rwanda,Published on 14 11 2022,Job,RW,"Accounting, Finance and investment, Other",Bachelor,Mid career (3 to 5 years of experience),Full-time,Sunday 20/11/2022 23:59,1
1,Production of Radio Drama Series for Behavior ...,UN Women Rwanda,Published on 14 11 2022,Consultancy,RW,Other,Bachelor,Senior (5+ years of experience),Contract,Monday 21/11/2022 09:00,1
2,Hiring a Firm for BRD Maintenance Assessment,Development Bank of Rwanda BRD,Published on 10 11 2022,Consultancy,RW,Other,Not specified,Not specified,Contract,Friday 25/11/2022 10:00,1
3,Construction du Guichet de Byumba,Reseau Interdiocesain de Microfinance RIM Ltd,Published on 10 11 2022,Tender,RW,Other,Not specified,Not specified,Tender,Monday 21/11/2022 23:59,1
4,Terms of Reference for Production Of the Secon...,UN Women Rwanda,Published on 02 11 2022,Consultancy,RW,"Economics, Other, Social sciences",Master,Senior (5+ years of experience),Contract,Tuesday 15/11/2022 17:00,1
...,...,...,...,...,...,...,...,...,...,...,...
152,Supply of Laboratory Reagents Equipments and ...,Rwanda Medical Supply Ltd Kigali,Published on 19 10 2022,Tender,Kigali RW,Other,Not specified,Not specified,Tender,Tuesday 15/11/2022 10:00,1
153,Rwanda Market Access and SRF MEL Associate,One Acre Fund Kigali,Published on 13 10 2022,Job,Kigali RW,"Agriculture, Business, Demography and data ana...",Bachelor,Entry level (1 to 3 years of experience),Full-time,Tuesday 10/01/2023 23:59,1
154,Consultant to Conduct a Baseline Survey for Ev...,Tubibe Amahoro TA,Published on 03 10 2022,Consultancy,RW,Other,Not specified,Not specified,Contract,Tuesday 15/11/2022 10:00,1
155,Rwanda Potato Seed Venture Lead,One Acre Fund Kigali,Published on 14 09 2022,Job,Kigali RW,"Agriculture, Agronomy, Business, Environmental...",Master,Senior (5+ years of experience),Full-time,Monday 28/11/2022 23:59,1


## Save Data to CSV File

In [ ]:
with open('jobs.csv', 'w+') as jobs_data:
  frame.to_csv(jobs_data)

# Challenge #3

## Get ***IT/Software*** Opportunities using `Regex`




In [ ]:
fr = df.filter(regex=".*Sector.*")
regex = r'Computer and IT'
it_df = frame[frame.Sector.str.match(regex)]

it_df

,Job Title,Company,Published Date,Job Category,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions
41,Product Manager,Ampersand Rwanda Ltd Kigali,Published on 10 11 2022,Job,Kigali RW,"Computer and IT, Management, Project management",Bachelor,Senior (5+ years of experience),Full-time,Tuesday 10/01/2023 23:59,1
120,IT and MIS Director,CHANCEN International Rwanda Kigali,Published on 31 10 2022,Job,Kigali RW,"Computer and IT, Other, Project management",Bachelor,Senior (5+ years of experience),Full-time,Friday 18/11/2022 23:59,1
131,Instructional Technology Assistant,University of Global Health Equity UGHE,Published on 28 10 2022,Job,RW,"Computer and IT, Education, Other",Bachelor,Not specified,Full-time,Monday 28/11/2022 23:59,1


In [ ]:
it_jobs = [dic for dic in dicts if 'Computer and IT' in dic['Sector'].split(',') ]

for i,v in enumerate(it_jobs[0].items()):
  print(f'{i} ===>{": ".join(v)}')


In [ ]:
dicts

In [ ]:
import json

result = it_df.to_json(orient='split')
result = json.loads(result)
# result = json.dumps(result, indent=4)

print(result["data"])

[['Product Manager', 'Ampersand Rwanda Ltd     Kigali', 'Published on 10 11 2022', 'Job', 'Kigali RW', 'Computer and IT, Management, Project management', 'Bachelor', 'Senior (5+ years of experience)', 'Full-time', 'Tuesday 10/01/2023 23:59', '1'], ['IT and MIS Director', 'CHANCEN International Rwanda     Kigali', 'Published on 31 10 2022', 'Job', 'Kigali RW', 'Computer and IT, Other, Project management', 'Bachelor', 'Senior (5+ years of experience)', 'Full-time', 'Friday 18/11/2022 23:59', '1'], ['Instructional Technology Assistant', 'University of Global Health Equity  UGHE', 'Published on 28 10 2022', 'Job', ' RW', 'Computer and IT, Education, Other', 'Bachelor', 'Not specified', 'Full-time', 'Monday 28/11/2022 23:59', '1']]


## Translate IT Jobs

In [ ]:
!pip install -U easynmt

In [ ]:
from easynmt import EasyNMT

model = EasyNMT('m2m_100_418M')

def translate(txt, lang):
  tran = model.translate(txt,source_lang='en', target_lang=lang)
  return tran


In [ ]:
[translate([': '.join(jo) for jo in job.items()], 'fr') for job in it_jobs]

# Challenge #4

# Challenge #5

## Host the API for the data in colab

In [ ]:
!ls

jobs.csv  sample_data


In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

!pip install -U easynmt

In [ ]:
import os
!mkdir fastapi_app/
os.chdir('fastapi_app')
!mkdir templates/
!mkdir static/

In [ ]:
%%writefile static/index.css
@import url(//fonts.googleapis.com/css?family=Lato:300:400);

body {
  margin:0;
}

h1 {
  font-family: 'Lato', sans-serif;
  font-weight:300;
  letter-spacing: 2px;
  font-size:48px;
}
p {
  font-family: 'Lato', sans-serif;
  letter-spacing: 1px;
  font-size:14px;
  color: #333333;
}

.header {
  position:relative;
  text-align:center;
  background: linear-gradient(60deg, rgba(84,58,183,1) 0%, rgba(0,172,193,1) 100%);
  color:white;
}
.logo {
  width:50px;
  fill:white;
  padding-right:15px;
  display:inline-block;
  vertical-align: middle;
}

.inner-header {
  height:65vh;
  width:100%;
  margin: 0;
  padding: 0;
}

.flex { /*Flexbox for containers*/
  display: flex;
  justify-content: center;
  align-items: center;
  text-align: center;
}

.waves {
  position:relative;
  width: 100%;
  height:15vh;
  margin-bottom:-7px; /*Fix for safari gap*/
  min-height:100px;
  max-height:150px;
}

.content {
  position:relative;
  height:20vh;
  text-align:center;
  background-color: white;
}

/* Animation */

.parallax > use {
  animation: move-forever 25s cubic-bezier(.55,.5,.45,.5)     infinite;
}
.parallax > use:nth-child(1) {
  animation-delay: -2s;
  animation-duration: 7s;
}
.parallax > use:nth-child(2) {
  animation-delay: -3s;
  animation-duration: 10s;
}
.parallax > use:nth-child(3) {
  animation-delay: -4s;
  animation-duration: 13s;
}
.parallax > use:nth-child(4) {
  animation-delay: -5s;
  animation-duration: 20s;
}
@keyframes move-forever {
  0% {
   transform: translate3d(-90px,0,0);
  }
  100% { 
    transform: translate3d(85px,0,0);
  }
}
/*Shrinking for mobile*/
@media (max-width: 768px) {
  .waves {
    height:40px;
    min-height:40px;
  }
  .content {
    height:30vh;
  }
  h1 {
    font-size:24px;
  }
}

Writing static/index.css


In [ ]:
!ls && pwd

static	templates
/content/fastapi_app


In [ ]:
%%writefile templates/index.html
<html>
  <head>
    <title>Hackerthon</title>
    <link href="{{ url_for('static', path='/index.css') }}" rel="stylesheet">
  </head>
  <body>
  <!--Hey! This is the original versionof Simple CSS Waves-->

<div class="header">

<!--Content before waves-->
<div class="inner-header flex">
<!--Just the logo.. Don't mind this-->
<svg version="1.1" class="logo" baseProfile="tiny" id="Layer_1" xmlns="http://www.w3.org/2000/svg"
xmlns:xlink="http://www.w3.org/1999/xlink" x="0px" y="0px" viewBox="0 0 500 500" xml:space="preserve">
<path fill="#FFFFFF" stroke="#000000" stroke-width="10" stroke-miterlimit="10" d="M57,283" />
<g><path fill="#fff"
d="M250.4,0.8C112.7,0.8,1,112.4,1,250.2c0,137.7,111.7,249.4,249.4,249.4c137.7,0,249.4-111.7,249.4-249.4
C499.8,112.4,388.1,0.8,250.4,0.8z M383.8,326.3c-62,0-101.4-14.1-117.6-46.3c-17.1-34.1-2.3-75.4,13.2-104.1
c-22.4,3-38.4,9.2-47.8,18.3c-11.2,10.9-13.6,26.7-16.3,45c-3.1,20.8-6.6,44.4-25.3,62.4c-19.8,19.1-51.6,26.9-100.2,24.6l1.8-39.7		c35.9,1.6,59.7-2.9,70.8-13.6c8.9-8.6,11.1-22.9,13.5-39.6c6.3-42,14.8-99.4,141.4-99.4h41L333,166c-12.6,16-45.4,68.2-31.2,96.2	c9.2,18.3,41.5,25.6,91.2,24.2l1.1,39.8C390.5,326.2,387.1,326.3,383.8,326.3z" />
</g>
</svg>
<h1>Hackathon NLP Fellowship</h1>
</div>
<div>
<a href="{{url}}/docs">API Docs</a>
</div>
<!--Waves Container-->
<div>
<svg class="waves" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"
viewBox="0 24 150 28" preserveAspectRatio="none" shape-rendering="auto">
<defs>
<path id="gentle-wave" d="M-160 44c30 0 58-18 88-18s 58 18 88 18 58-18 88-18 58 18 88 18 v44h-352z" />
</defs>
<g class="parallax">
<use xlink:href="#gentle-wave" x="48" y="0" fill="rgba(255,255,255,0.7" />
<use xlink:href="#gentle-wave" x="48" y="3" fill="rgba(255,255,255,0.5)" />
<use xlink:href="#gentle-wave" x="48" y="5" fill="rgba(255,255,255,0.3)" />
<use xlink:href="#gentle-wave" x="48" y="7" fill="#fff" />
</g>
</svg>
</div>
<!--Waves end-->

</div>
<!--Header ends-->

<!--Content starts-->
<div class="content flex">
  <p>&copy;By.Josue BATEY | All right reserved </p>
</div>
<!--Content ends-->
 </body>
</html

Overwriting templates/index.html


## Site Variable

In [ ]:

langages = {
  'aa': 'Afar',
  'ab': 'Abkhazian',
  'af': 'Afrikaans',
  'ak': 'Akan',
  'sq': 'Albanian',
  'am': 'Amharic',
  'ar': 'Arabic',
  'an': 'Aragonese',
  'hy': 'Armenian',
  'as': 'Assamese',
  'av': 'Avaric',
  'ae': 'Avestan',
  'ay': 'Aymara',
  'az': 'Azerbaijani',
  'ba': 'Bashkir',
  'bm': 'Bambara',
  'eu': 'Basque',
  'be': 'Belarusian',
  'bn': 'Bengali',
  'bh': 'Bihari languages',
  'bi': 'Bislama',
  'bo': 'Tibetan',
  'bs': 'Bosnian',
  'br': 'Breton',
  'bg': 'Bulgarian',
  'my': 'Burmese',
  'ca': 'Catalan; Valencian',
  'cs': 'Czech',
  'ch': 'Chamorro',
  'ce': 'Chechen',
  'zh': 'Chinese',
  'cu': 'Church Slavic; Old Slavonic; Church Slavonic; Old Bulgarian; Old Church Slavonic',
  'cv': 'Chuvash',
  'kw': 'Cornish',
  'co': 'Corsican',
  'cr': 'Cree',
  'cy': 'Welsh',
  'cs': 'Czech',
  'da': 'Danish',
  'de': 'German',
  'dv': 'Divehi; Dhivehi; Maldivian',
  'nl': 'Dutch; Flemish',
  'dz': 'Dzongkha',
  'el': 'Greek, Modern (1453-)',
  'en': 'English',
  'eo': 'Esperanto',
  'et': 'Estonian',
  'eu': 'Basque',
  'ee': 'Ewe',
  'fo': 'Faroese',
  'fa': 'Persian',
  'fj': 'Fijian',
  'fi': 'Finnish',
  'fr': 'French',
  'fy': 'Western Frisian',
  'ff': 'Fulah',
  'Ga': 'Georgian',
  'de': 'German',
  'gd': 'Gaelic; Scottish Gaelic',
  'ga': 'Irish',
  'gl': 'Galician',
  'gv': 'Manx',
  'el': 'Greek, Modern (1453-)',
  'gn': 'Guarani',
  'gu': 'Gujarati',
  'ht': 'Haitian; Haitian Creole',
  'ha': 'Hausa',
  'he': 'Hebrew',
  'hz': 'Herero',
  'hi': 'Hindi',
  'ho': 'Hiri Motu',
  'hr': 'Croatian',
  'hu': 'Hungarian',
  'hy': 'Armenian',
  'ig': 'Igbo',
  'is': 'Icelandic',
  'io': 'Ido',
  'ii': 'Sichuan Yi; Nuosu',
  'iu': 'Inuktitut',
  'ie': 'Interlingue; Occidental',
  'ia': 'Interlingua (International Auxiliary Language Association)',
  'id': 'Indonesian',
  'ik': 'Inupiaq',
  'is': 'Icelandic',
  'it': 'Italian',
  'jv': 'Javanese',
  'ja': 'Japanese',
  'kl': 'Kalaallisut; Greenlandic',
  'kn': 'Kannada',
  'ks': 'Kashmiri',
  'ka': 'Georgian',
  'kr': 'Kanuri',
  'kk': 'Kazakh',
  'km': 'Central Khmer',
  'ki': 'Kikuyu; Gikuyu',
  'rw': 'Kinyarwanda',
  'ky': 'Kirghiz; Kyrgyz',
  'kv': 'Komi',
  'kg': 'Kongo',
  'ko': 'Korean',
  'kj': 'Kuanyama; Kwanyama',
  'ku': 'Kurdish',
  'lo': 'Lao',
  'la': 'Latin',
  'lv': 'Latvian',
  'li': 'Limburgan; Limburger; Limburgish',
  'ln': 'Lingala',
  'lt': 'Lithuanian',
  'lb': 'Luxembourgish; Letzeburgesch',
  'lu': 'Luba-Katanga',
  'lg': 'Ganda',
  'mk': 'Macedonian',
  'mh': 'Marshallese',
  'ml': 'Malayalam',
  'mi': 'Maori',
  'mr': 'Marathi',
  'ms': 'Malay',
  'Mi': 'Micmac',
  'mk': 'Macedonian',
  'mg': 'Malagasy',
  'mt': 'Maltese',
  'mn': 'Mongolian',
  'mi': 'Maori',
  'ms': 'Malay',
  'my': 'Burmese',
  'na': 'Nauru',
  'nv': 'Navajo; Navaho',
  'nr': 'Ndebele, South; South Ndebele',
  'nd': 'Ndebele, North; North Ndebele',
  'ng': 'Ndonga',
  'ne': 'Nepali',
  'nl': 'Dutch; Flemish',
  'nn': 'Norwegian Nynorsk; Nynorsk, Norwegian',
  'nb': 'Bokmål, Norwegian; Norwegian Bokmål',
  'no': 'Norwegian',
  'oc': 'Occitan (post 1500)',
  'oj': 'Ojibwa',
  'or': 'Oriya',
  'om': 'Oromo',
  'os': 'Ossetian; Ossetic',
  'pa': 'Panjabi; Punjabi',
  'fa': 'Persian',
  'pi': 'Pali',
  'pl': 'Polish',
  'pt': 'Portuguese',
  'ps': 'Pushto; Pashto',
  'qu': 'Quechua',
  'rm': 'Romansh',
  'ro': 'Romanian; Moldavian; Moldovan',
  'ro': 'Romanian; Moldavian; Moldovan',
  'rn': 'Rundi',
  'ru': 'Russian',
  'sg': 'Sango',
  'sa': 'Sanskrit',
  'si': 'Sinhala; Sinhalese',
  'sk': 'Slovak',
  'sk': 'Slovak',
  'sl': 'Slovenian',
  'se': 'Northern Sami',
  'sm': 'Samoan',
  'sn': 'Shona',
  'sd': 'Sindhi',
  'so': 'Somali',
  'st': 'Sotho, Southern',
  'es': 'Spanish; Castilian',
  'sq': 'Albanian',
  'sc': 'Sardinian',
  'sr': 'Serbian',
  'ss': 'Swati',
  'su': 'Sundanese',
  'sw': 'Swahili',
  'sv': 'Swedish',
  'ty': 'Tahitian',
  'ta': 'Tamil',
  'tt': 'Tatar',
  'te': 'Telugu',
  'tg': 'Tajik',
  'tl': 'Tagalog',
  'th': 'Thai',
  'bo': 'Tibetan',
  'ti': 'Tigrinya',
  'to': 'Tonga (Tonga Islands)',
  'tn': 'Tswana',
  'ts': 'Tsonga',
  'tk': 'Turkmen',
  'tr': 'Turkish',
  'tw': 'Twi',
  'ug': 'Uighur; Uyghur',
  'uk': 'Ukrainian',
  'ur': 'Urdu',
  'uz': 'Uzbek',
  've': 'Venda',
  'vi': 'Vietnamese',
  'vo': 'Volapük',
  'cy': 'Welsh',
  'wa': 'Walloon',
  'wo': 'Wolof',
  'xh': 'Xhosa',
  'yi': 'Yiddish',
  'yo': 'Yoruba',
  'za': 'Zhuang; Chuang',
  'zh': 'Chinese',
  'zu': 'Zulu',
}

## FASTAPI APP

In [ ]:
from fastapi import FastAPI, Request
from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse
from easynmt import EasyNMT

from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles

import random

app = FastAPI(title='Hackerthon Jobs API') 


app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")


model = EasyNMT('m2m_100_418M')

def translate_text(txt, lang):
  tran = model.translate(txt, target_lang = lang)
  return tran

@app.get('/')#ROUTE
def index(request: Request,lang:str = ''):
  return templates.TemplateResponse("index.html", context = {"request": request,"url": url} )

@app.get('/jobs')
def jobs():
  return dicts

@app.get('/jobs/it')
def jobs():
  return it_jobs

@app.get('/jobs/it/{lang}')
def jobs(lang:str = 'fr'):
  return [translate_text([': '.join(jo) for jo in job.items()], lang) for job in it_jobs]


In [ ]:
!pip install jinja2 python-multipart

In [ ]:
!ngrok authtoken 2HV7V6qaQVZB0haTh3ZDDj1d2Yf_6ybMdAJJFbDn1ziWnomBV

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [1]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(4994)
print("started...")
print("url ===>:", ngrok_tunnel.public_url)
url = ngrok_tunnel.public_url

nest_asyncio.apply()
uvicorn.run(app, port=4994)

ModuleNotFoundError: ignored

In [ ]:
# !pkill ngrok